# Reducing Customer Churn at Telco

---

## Project Goals

The goal of this project is to identify drivers of customer churn at Telco, produce a prediction model to identify 
which customers are at the highest risk of churning, and offer a recommendation for reducing customer churn.

---

## Project Description

Telco customers are churning at an unacceptably high rate which is affecting the company's bottom line. Retaining
existing customers costs far less than signing new customers. As such we want to reduce churn in order to help
the Telco's bottom line rather than relying on signing new customers to make up the difference. We will compare
and contrast customers who have churned versus those who haven't to determine the attributes that are driving 
customers to churn. We will produce a prediction model to help identify the customers that are at the highest risk 
of churning and we will provide a list of customers who are likely to churn (provided in predictions.csv). Finally, 
we will offer a recommended course of action to help promote customer retention.

---

# Imports

Below are all the imports needed to reproduce this project. This utilizes numpy, pandas, matplotlib, seaborn and sklearn. If you do not have these installed already install them now before proceeding.

In [4]:
import os
import numpy as np
import pandas as pd

# Data Acquisition

---

## Accessing the Database

Here we will outline the process for acquiring the Telco customer dataset from the MySQL database. In order to access this data you will need login credentials. Assuming you have credentials save these in a env.py file in the following form:

In [1]:
username = 'your_username'
password = 'your_password'
hostname = 'data.codeup.com'

Save this file in the notebooks directory and save a copy in the util directory for use with the final report notebook. We will import our credentials from env.py and utilize the following function to acquiring our data.

In [2]:
from env import username, password, hostname

def get_db_url(database_name, username = username, password = password, hostname = hostname):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'

---

## Acquiring the Data

We can gather all the data we need by using a SQL query that joins together all the tables in the telco_churn database. The telco_churn database has several tables related with foreign keys. Here is a function that will give us the required SQL query:

In [3]:
def get_telco_sql():
    return '''
        SELECT *
        FROM customers
        JOIN payment_types USING (payment_type_id)
        JOIN internet_service_types USING (internet_service_type_id)
        JOIN contract_types USING (contract_type_id);
    '''

Next we want to read the dataset from the database. We would also like to cache this data for quicker access in the future. If we are doing this we must check if the cached file already exists before we try to read it from the database. We can achieve this using pandas and the os module from the python standard library. We put all this code into a function for our convenience.

In [5]:
def get_telco_data(use_cache = True):
    # If the file is cached, read from the .csv file
    if os.path.exists('telco.csv') and use_cache:
        print('Using cache')
        return pd.read_csv('telco.csv')
    
    # Otherwise read from the mysql database
    else:
        print('Reading from database')
        df = pd.read_sql(get_telco_sql(), get_db_url('telco_churn'))
        df.to_csv('telco.csv', index = False)
        return df

In [6]:
# Now we can acquire the data needed to proceed
telco_customers = get_telco_data()
telco_customers.head(2)

Using cache


,contract_type_id,internet_service_type_id,payment_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,payment_type,internet_service_type,contract_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.6,593.3,No,Mailed check,DSL,One year
1,1,1,2,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.9,542.4,No,Mailed check,DSL,Month-to-month


---

